In [1]:
pip install mediapipe


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: C:\Users\sharm\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip


In [ ]:
import cv2
import mediapipe as mp
import numpy as np

# Initialize Mediapipe Hand Tracking
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.8)

# Drawing canvas and variables
canvas = np.zeros((480, 640, 3), dtype=np.uint8)
prev_x, prev_y = None, None
smooth_factor = 0.3  # Increased smoothness

# Colors
colors = {
    "red": (0, 0, 255),
    "blue": (255, 0, 0),
    "green": (0, 255, 0),
    "white": (255, 255, 255)
}
current_color = colors["white"]  # Default color
eraser_mode = False

def smooth_point(new, old):
    """Applies weighted smoothing for a more stable pointer."""
    if old is None:
        return new
    return int(smooth_factor * new + (1 - smooth_factor) * old)

def detect_gesture(landmarks):
    """Identifies user gestures for color selection, writing, and erasing."""
    global current_color, eraser_mode, canvas

    # Get landmark positions
    index_tip = landmarks[mp_hands.HandLandmark.INDEX_FINGER_TIP].y
    middle_tip = landmarks[mp_hands.HandLandmark.MIDDLE_FINGER_TIP].y
    ring_tip = landmarks[mp_hands.HandLandmark.RING_FINGER_TIP].y
    pinky_tip = landmarks[mp_hands.HandLandmark.PINKY_TIP].y
    thumb_tip = landmarks[mp_hands.HandLandmark.THUMB_TIP].x
    thumb_ip = landmarks[mp_hands.HandLandmark.THUMB_IP].x

    # Measure distance between fingers to confirm gestures
    def is_finger_up(finger_tip, other_finger_tip, threshold=0.02):
        return abs(finger_tip - other_finger_tip) > threshold

    # Gesture for Clearing Canvas (Closed Fist)
    if all(abs(landmarks[i].x - landmarks[i + 1].x) < 0.02 for i in range(4, 8)):
        canvas[:] = 0
        print("🗑️ Canvas Cleared!")

    # Gesture for Color Selection
    if is_finger_up(pinky_tip, ring_tip):
        current_color = colors["red"]
        print("🎨 Color Changed: Red")

    elif is_finger_up(ring_tip, middle_tip):
        current_color = colors["blue"]
        print("🎨 Color Changed: Blue")

    elif is_finger_up(middle_tip, index_tip):
        current_color = colors["green"]
        print("🎨 Color Changed: Green")

    # Writing Mode (Index Finger is up)
    if is_finger_up(index_tip, middle_tip):
        print("✏️ Writing Mode")

    # Eraser Mode (Thumb touching index finger)
    if abs(thumb_tip - thumb_ip) < 0.05:
        eraser_mode = True
        print("🧽 Eraser Mode Activated")
    else:
        eraser_mode = False

# Start Video Capture
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)  # Flip for natural interaction
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb_frame)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            detect_gesture(hand_landmarks.landmark)  # Detect gestures

            # Get index finger tip position
            index_finger = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
            x, y = int(index_finger.x * 640), int(index_finger.y * 480)

            # Apply smoothing
            x = smooth_point(x, prev_x)
            y = smooth_point(y, prev_y)

            if prev_x is not None and prev_y is not None:
                thickness = 10 if eraser_mode else 3  # Thicker for eraser
                color = (0, 0, 0) if eraser_mode else current_color  # Black for eraser
                
                cv2.line(canvas, (prev_x, prev_y), (x, y), color, thickness)

            prev_x, prev_y = x, y  # Update previous point

    # Overlay drawing on frame
    output = cv2.addWeighted(frame, 0.5, canvas, 0.5, 0)
    
    # Display the current color on screen
    cv2.rectangle(output, (10, 10), (50, 50), current_color, -1)

    cv2.imshow("Air Notepad", output)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [2]:
import cv2
import mediapipe as mp
import numpy as np

# Initialize Mediapipe Hand Tracking
mp_hands = mp.solutions.hands
mp_draw = mp.solutions.drawing_utils
hands = mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.7)

# Create a black canvas for drawing
canvas = np.zeros((480, 640, 3), dtype=np.uint8)

cap = cv2.VideoCapture(0)  # Open webcam

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Flip image and convert color (for better tracking)
    frame = cv2.flip(frame, 1)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the frame with Mediapipe
    result = hands.process(rgb_frame)
    
    if result.multi_hand_landmarks:
        for hand_landmarks in result.multi_hand_landmarks:
            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            # Extract index finger tip (Landmark 8)
            index_finger_tip = hand_landmarks.landmark[8]
            h, w, c = frame.shape
            cx, cy = int(index_finger_tip.x * w), int(index_finger_tip.y * h)

            # Draw on canvas
            cv2.circle(canvas, (cx, cy), 5, (0, 255, 0), -1)

    # Overlay canvas on frame
    combined = cv2.addWeighted(frame, 0.5, canvas, 0.5, 0)

    # Show Output
    cv2.imshow("Air Notepad", combined)

    # Press 'q' to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


AttributeError: module 'mediapipe' has no attribute 'solutions'

In [3]:
import cv2
import mediapipe as mp
import numpy as np

# Initialize Mediapipe Hand Tracking
mp_hands = mp.solutions.hands
mp_draw = mp.solutions.drawing_utils
hands = mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.7)

# Create a black canvas for drawing
canvas = np.zeros((480, 640, 3), dtype=np.uint8)

cap = cv2.VideoCapture(0)  # Open webcam

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Flip image and convert color (for better tracking)
    frame = cv2.flip(frame, 1)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the frame with Mediapipe
    result = hands.process(rgb_frame)
    
    if result.multi_hand_landmarks:
        for hand_landmarks in result.multi_hand_landmarks:
            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            # Extract index finger tip (Landmark 8)
            index_finger_tip = hand_landmarks.landmark[8]
            h, w, c = frame.shape
            cx, cy = int(index_finger_tip.x * w), int(index_finger_tip.y * h)

            # Draw on canvas
            cv2.circle(canvas, (cx, cy), 5, (0, 255, 0), -1)

    # Overlay canvas on frame
    combined = cv2.addWeighted(frame, 0.5, canvas, 0.5, 0)

    # Show Output
    cv2.imshow("Air Notepad", combined)

    # Press 'q' to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


AttributeError: module 'mediapipe' has no attribute 'solutions'

In [2]:
import cv2
import mediapipe as mp
import numpy as np

# Initialize Mediapipe Hand Tracking
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    min_detection_confidence=0.8,
    min_tracking_confidence=0.8,
    max_num_hands=2  # Track both hands
)

# Canvas settings
canvas_width, canvas_height = 640, 480
canvas = np.zeros((canvas_height, canvas_width, 3), dtype=np.uint8)
prev_x, prev_y = None, None
smooth_factor = 0.3

# Menu settings
menu_width = 120
menu_visible = True

# Tool settings
class Tool:
    PEN = "pen"
    BRUSH = "brush"
    ERASER = "eraser"

# Colors with labels
color_palette = [
    ("Red", (0, 0, 255)),
    ("Blue", (255, 0, 0)),
    ("Green", (0, 255, 0)),
    ("Yellow", (0, 255, 255)),
    ("Purple", (255, 0, 255)),
    ("White", (255, 255, 255)),
    ("Orange", (0, 165, 255)),
    ("Pink", (203, 192, 255))
]

# Current settings
current_tool = Tool.PEN
current_color = (255, 255, 255)  # White
current_color_name = "White"

# Tool properties
tool_properties = {
    Tool.PEN: {"thickness": 3, "alpha": 1.0},
    Tool.BRUSH: {"thickness": 8, "alpha": 0.7},
    Tool.ERASER: {"thickness": 20, "alpha": 1.0}
}

def draw_menu(frame):
    """Draw the interactive menu on the left side of the screen."""
    menu_bg = np.ones((canvas_height, menu_width, 3), dtype=np.uint8) * 40
    
    y_offset = 20
    
    # Title
    cv2.putText(menu_bg, "TOOLS", (10, y_offset), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
    y_offset += 30
    
    # Tool buttons
    tools = [
        (Tool.PEN, "Pen"),
        (Tool.BRUSH, "Brush"),
        (Tool.ERASER, "Eraser")
    ]
    
    tool_buttons = []
    for tool_id, tool_name in tools:
        button_color = (100, 200, 100) if current_tool == tool_id else (80, 80, 80)
        cv2.rectangle(menu_bg, (10, y_offset), (110, y_offset + 30), button_color, -1)
        cv2.rectangle(menu_bg, (10, y_offset), (110, y_offset + 30), (200, 200, 200), 2)
        cv2.putText(menu_bg, tool_name, (20, y_offset + 20),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1)
        tool_buttons.append((10, y_offset, 110, y_offset + 30, tool_id))
        y_offset += 40
    
    y_offset += 20
    cv2.putText(menu_bg, "COLORS", (10, y_offset),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
    y_offset += 30
    
    # Color palette
    color_buttons = []
    for i, (name, color) in enumerate(color_palette):
        row = i // 2
        col = i % 2
        x = 10 + col * 55
        y = y_offset + row * 45
        
        # Highlight selected color
        if name == current_color_name:
            cv2.rectangle(menu_bg, (x-3, y-3), (x+43, y+43), (255, 255, 0), 3)
        
        cv2.rectangle(menu_bg, (x, y), (x+40, y+40), color, -1)
        cv2.rectangle(menu_bg, (x, y), (x+40, y+40), (200, 200, 200), 2)
        color_buttons.append((x, y, x+40, y+40, name, color))
    
    # Clear button at bottom
    clear_y = canvas_height - 50
    cv2.rectangle(menu_bg, (10, clear_y), (110, clear_y + 35), (0, 0, 150), -1)
    cv2.rectangle(menu_bg, (10, clear_y), (110, clear_y + 35), (200, 200, 200), 2)
    cv2.putText(menu_bg, "CLEAR", (25, clear_y + 23),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
    clear_button = (10, clear_y, 110, clear_y + 35, "clear")
    
    # Combine menu with frame
    frame[0:canvas_height, 0:menu_width] = menu_bg
    
    return tool_buttons, color_buttons, clear_button

def check_menu_interaction(x, y, tool_buttons, color_buttons, clear_button):
    """Check if a point intersects with any menu button."""
    global current_tool, current_color, current_color_name, canvas
    
    # Check tool buttons
    for x1, y1, x2, y2, tool_id in tool_buttons:
        if x1 <= x <= x2 and y1 <= y <= y2:
            current_tool = tool_id
            print(f"🛠️ Tool changed to: {tool_id}")
            return True
    
    # Check color buttons
    for x1, y1, x2, y2, name, color in color_buttons:
        if x1 <= x <= x2 and y1 <= y <= y2:
            current_color = color
            current_color_name = name
            print(f"🎨 Color changed to: {name}")
            return True
    
    # Check clear button
    x1, y1, x2, y2, _ = clear_button
    if x1 <= x <= x2 and y1 <= y <= y2:
        canvas[:] = 0
        print("🗑️ Canvas cleared!")
        return True
    
    return False

def smooth_point(new, old):
    """Apply smoothing to pointer movement."""
    if old is None:
        return new
    return int(smooth_factor * new + (1 - smooth_factor) * old)

def detect_pinch(hand_landmarks):
    """Detect pinch gesture (thumb and index finger touching)."""
    thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]
    index_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
    
    distance = np.sqrt(
        (thumb_tip.x - index_tip.x)**2 + 
        (thumb_tip.y - index_tip.y)**2
    )
    
    return distance < 0.05  # Threshold for pinch detection

def get_hand_label(hand_landmarks, hand_handedness):
    
    return hand_handedness.classification[0].label

# Start video capture
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, canvas_width)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, canvas_height)

left_hand_pinched = False
right_hand_drawing = False

print("Air Notepad Started!")
print("Use LEFT hand to select tools/colors (pinch gesture)")
print("Use RIGHT hand to draw (index finger extended)")
print("Press 'q' to quit")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    frame = cv2.flip(frame, 1)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb_frame)
    
    # Draw menu and get button positions
    tool_buttons, color_buttons, clear_button = draw_menu(frame)
    
    if results.multi_hand_landmarks and results.multi_handedness:
        for hand_landmarks, handedness in zip(results.multi_hand_landmarks, results.multi_handedness):
            hand_label = get_hand_label(hand_landmarks, handedness)
            
            # Get index finger position
            index_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
            x = int(index_tip.x * canvas_width)
            y = int(index_tip.y * canvas_height)
            
            # LEFT HAND: Menu interaction with pinch gesture
            if hand_label == "Left":
                is_pinching = detect_pinch(hand_landmarks)
                
                # Draw cursor for left hand
                cursor_color = (0, 255, 255) if is_pinching else (0, 200, 200)
                cv2.circle(frame, (x, y), 8, cursor_color, -1)
                cv2.circle(frame, (x, y), 10, (255, 255, 255), 2)
                
                # Check menu interaction when pinching
                if is_pinching and not left_hand_pinched:
                    check_menu_interaction(x, y, tool_buttons, color_buttons, clear_button)
                    left_hand_pinched = True
                elif not is_pinching:
                    left_hand_pinched = False
            
            # RIGHT HAND: Drawing
            elif hand_label == "Right":
                # Check if index finger is extended (for drawing)
                index_tip_y = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y
                middle_tip_y = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP].y
                index_extended = index_tip_y < middle_tip_y - 0.03
                
                # Draw cursor for right hand
                cursor_color = (0, 255, 0) if index_extended else (100, 100, 100)
                cv2.circle(frame, (x, y), 6, cursor_color, -1)
                
                # Only draw if outside menu area and index finger is extended
                if x > menu_width and index_extended:
                    x_smooth = smooth_point(x, prev_x)
                    y_smooth = smooth_point(y, prev_y)
                    
                    if prev_x is not None and prev_y is not None and prev_x > menu_width:
                        props = tool_properties[current_tool]
                        thickness = props["thickness"]
                        
                        if current_tool == Tool.ERASER:
                            color = (0, 0, 0)
                        else:
                            color = current_color
                        
                        cv2.line(canvas, (prev_x, prev_y), (x_smooth, y_smooth), color, thickness)
                    
                    prev_x, prev_y = x_smooth, y_smooth
                else:
                    prev_x, prev_y = None, None
    else:
        prev_x, prev_y = None, None
    
    # Overlay canvas on frame
    output = cv2.addWeighted(frame, 0.6, canvas, 0.4, 0)
    
    # Add status bar
    status_text = f"Tool: {current_tool.upper()} | Color: {current_color_name}"
    cv2.rectangle(output, (menu_width, 0), (canvas_width, 30), (50, 50, 50), -1)
    cv2.putText(output, status_text, (menu_width + 10, 20),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
    
    cv2.imshow("Air Notepad", output)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
print("Air Notepad closed!")

Air Notepad Started!
Use LEFT hand to select tools/colors (pinch gesture)
Use RIGHT hand to draw (index finger extended)
Press 'q' to quit
🛠️ Tool changed to: eraser
🎨 Color changed to: Pink
🗑️ Canvas cleared!
🛠️ Tool changed to: brush
🎨 Color changed to: Yellow
🎨 Color changed to: Blue
🎨 Color changed to: Blue
🗑️ Canvas cleared!
🎨 Color changed to: Yellow
🗑️ Canvas cleared!
🗑️ Canvas cleared!
🗑️ Canvas cleared!
🗑️ Canvas cleared!
Air Notepad closed!


In [ ]:
import cv2
import mediapipe as mp
import numpy as np

# Initialize Mediapipe Hand Tracking
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    min_detection_confidence=0.8,
    min_tracking_confidence=0.8,
    max_num_hands=2  # Track both hands
)

# Canvas settings - increased size
canvas_width, canvas_height = 1280, 720
canvas = np.zeros((canvas_height, canvas_width, 3), dtype=np.uint8)
prev_x, prev_y = None, None
smooth_factor = 0.3

# Menu settings
menu_width = 120
menu_visible = True

# Tool settings
class Tool:
    PEN = "pen"
    BRUSH = "brush"
    ERASER = "eraser"

# Colors with labels
color_palette = [
    ("Red", (0, 0, 255)),
    ("Blue", (255, 0, 0)),
    ("Green", (0, 255, 0)),
    ("Yellow", (0, 255, 255)),
    ("Purple", (255, 0, 255)),
    ("White", (255, 255, 255)),
    ("Orange", (0, 165, 255)),
    ("Pink", (203, 192, 255))
]

# Current settings
current_tool = Tool.PEN
current_color = (255, 255, 255)  # White
current_color_name = "White"

# Tool properties
tool_properties = {
    Tool.PEN: {"thickness": 3, "alpha": 1.0},
    Tool.BRUSH: {"thickness": 8, "alpha": 0.7},
    Tool.ERASER: {"thickness": 20, "alpha": 1.0}
}

def draw_menu(frame):
    """Draw the interactive menu on the left side of the screen."""
    menu_bg = np.ones((canvas_height, menu_width, 3), dtype=np.uint8) * 40
    
    y_offset = 20
    
    # Title
    cv2.putText(menu_bg, "TOOLS", (10, y_offset), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
    y_offset += 30
    
    # Tool buttons
    tools = [
        (Tool.PEN, "Pen"),
        (Tool.BRUSH, "Brush"),
        (Tool.ERASER, "Eraser")
    ]
    
    tool_buttons = []
    for tool_id, tool_name in tools:
        button_color = (100, 200, 100) if current_tool == tool_id else (80, 80, 80)
        cv2.rectangle(menu_bg, (10, y_offset), (110, y_offset + 30), button_color, -1)
        cv2.rectangle(menu_bg, (10, y_offset), (110, y_offset + 30), (200, 200, 200), 2)
        cv2.putText(menu_bg, tool_name, (20, y_offset + 20),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1)
        tool_buttons.append((10, y_offset, 110, y_offset + 30, tool_id))
        y_offset += 40
    
    y_offset += 20
    cv2.putText(menu_bg, "COLORS", (10, y_offset),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
    y_offset += 30
    
    # Color palette
    color_buttons = []
    for i, (name, color) in enumerate(color_palette):
        row = i // 2
        col = i % 2
        x = 10 + col * 55
        y = y_offset + row * 45
        
        # Highlight selected color
        if name == current_color_name:
            cv2.rectangle(menu_bg, (x-3, y-3), (x+43, y+43), (255, 255, 0), 3)
        
        cv2.rectangle(menu_bg, (x, y), (x+40, y+40), color, -1)
        cv2.rectangle(menu_bg, (x, y), (x+40, y+40), (200, 200, 200), 2)
        color_buttons.append((x, y, x+40, y+40, name, color))
    
    # Clear button right below colors
    y_offset += (len(color_palette) // 2) * 45 + 20
    cv2.rectangle(menu_bg, (10, y_offset), (110, y_offset + 35), (0, 0, 150), -1)
    cv2.rectangle(menu_bg, (10, y_offset), (110, y_offset + 35), (200, 200, 200), 2)
    cv2.putText(menu_bg, "CLEAR", (25, y_offset + 23),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
    clear_button = (10, y_offset, 110, y_offset + 35, "clear")
    
    # Combine menu with frame
    frame[0:canvas_height, 0:menu_width] = menu_bg
    
    return tool_buttons, color_buttons, clear_button

def check_menu_interaction(x, y, tool_buttons, color_buttons, clear_button):
    """Check if a point intersects with any menu button."""
    global current_tool, current_color, current_color_name, canvas
    
    # Check tool buttons
    for x1, y1, x2, y2, tool_id in tool_buttons:
        if x1 <= x <= x2 and y1 <= y <= y2:
            current_tool = tool_id
            print(f"🛠️ Tool changed to: {tool_id}")
            return True
    
    # Check color buttons
    for x1, y1, x2, y2, name, color in color_buttons:
        if x1 <= x <= x2 and y1 <= y <= y2:
            current_color = color
            current_color_name = name
            print(f"🎨 Color changed to: {name}")
            return True
    
    # Check clear button
    x1, y1, x2, y2, _ = clear_button
    if x1 <= x <= x2 and y1 <= y <= y2:
        canvas[:] = 0
        print("🗑️ Canvas cleared!")
        return True
    
    return False

def smooth_point(new, old):
    """Apply smoothing to pointer movement."""
    if old is None:
        return new
    return int(smooth_factor * new + (1 - smooth_factor) * old)

def detect_pinch(hand_landmarks):
    """Detect pinch gesture (thumb and index finger touching)."""
    thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]
    index_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
    
    distance = np.sqrt(
        (thumb_tip.x - index_tip.x)**2 + 
        (thumb_tip.y - index_tip.y)**2
    )
    
    return distance < 0.05  # Threshold for pinch detection

def get_hand_label(hand_landmarks, hand_handedness):
    
    return hand_handedness.classification[0].label

# Start video capture
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, canvas_width)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, canvas_height)

left_hand_pinched = False
right_hand_drawing = False

print("Air Notepad Started!")
print("Use LEFT hand to select tools/colors (pinch gesture)")
print("Use RIGHT hand to draw (index finger extended)")
print("Press 'q' to quit")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    frame = cv2.flip(frame, 1)
    frame = cv2.resize(frame, (canvas_width, canvas_height))
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb_frame)
    
    # Draw menu and get button positions
    tool_buttons, color_buttons, clear_button = draw_menu(frame)
    
    if results.multi_hand_landmarks and results.multi_handedness:
        for hand_landmarks, handedness in zip(results.multi_hand_landmarks, results.multi_handedness):
            hand_label = get_hand_label(hand_landmarks, handedness)
            
            # Get index finger position
            index_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
            x = int(index_tip.x * canvas_width)
            y = int(index_tip.y * canvas_height)
            
            # LEFT HAND: Menu interaction with pinch gesture
            if hand_label == "Left":
                is_pinching = detect_pinch(hand_landmarks)
                
                # Draw cursor for left hand
                cursor_color = (0, 255, 255) if is_pinching else (0, 200, 200)
                cv2.circle(frame, (x, y), 8, cursor_color, -1)
                cv2.circle(frame, (x, y), 10, (255, 255, 255), 2)
                
                # Check menu interaction when pinching
                if is_pinching and not left_hand_pinched:
                    check_menu_interaction(x, y, tool_buttons, color_buttons, clear_button)
                    left_hand_pinched = True
                elif not is_pinching:
                    left_hand_pinched = False
            
            # RIGHT HAND: Drawing
            elif hand_label == "Right":
                # Check if index finger is extended (for drawing)
                index_tip_y = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y
                middle_tip_y = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP].y
                index_extended = index_tip_y < middle_tip_y - 0.03
                
                # Draw cursor for right hand
                cursor_color = (0, 255, 0) if index_extended else (100, 100, 100)
                cv2.circle(frame, (x, y), 6, cursor_color, -1)
                
                # Only draw if outside menu area and index finger is extended
                if x > menu_width and index_extended:
                    x_smooth = smooth_point(x, prev_x)
                    y_smooth = smooth_point(y, prev_y)
                    
                    if prev_x is not None and prev_y is not None and prev_x > menu_width:
                        props = tool_properties[current_tool]
                        thickness = props["thickness"]
                        
                        if current_tool == Tool.ERASER:
                            color = (0, 0, 0)
                        else:
                            color = current_color
                        
                        cv2.line(canvas, (prev_x, prev_y), (x_smooth, y_smooth), color, thickness)
                    
                    prev_x, prev_y = x_smooth, y_smooth
                else:
                    prev_x, prev_y = None, None
    else:
        prev_x, prev_y = None, None
    
    # Overlay canvas on frame
    output = cv2.addWeighted(frame, 0.6, canvas, 0.4, 0)
    
    # Add status bar
    status_text = f"Tool: {current_tool.upper()} | Color: {current_color_name}"
    cv2.rectangle(output, (menu_width, 0), (canvas_width, 30), (50, 50, 50), -1)
    cv2.putText(output, status_text, (menu_width + 10, 20),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
    
    cv2.imshow("Air Notepad", output)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
print("Air Notepad closed!")

Air Notepad Started!
Use LEFT hand to select tools/colors (pinch gesture)
Use RIGHT hand to draw (index finger extended)
Press 'q' to quit
🛠️ Tool changed to: pen
🎨 Color changed to: Yellow
🎨 Color changed to: White
🗑️ Canvas cleared!
